In [6]:
import numpy as np
import h5py
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
import numpy as np
import scipy.optimize as opt
import sys, os, random, gzip
import tensorflow as tf
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import adam
from keras.losses import categorical_crossentropy
import keras
import matplotlib.pyplot as plt
import h5py
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
import sys
from sklearn import metrics
from keras.models import load_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as skAUC
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, TensorBoard
import tensorflow as tf
from keras import backend as K
import time
import glob
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc


Using TensorFlow backend.


In [31]:
f = h5py.File("data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z",'r',track_order=True)
f.keys()

<KeysViewHDF5 ['t_allpar_new']>

In [32]:
treeArray = f['t_allpar_new'][()]
print(treeArray.dtype.names)

('index', 'j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_pdgid', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel', 'j1_e1mcosthetarel', 'j_g', 'j_q', 

In [33]:
import itertools
x = [len(list(y)) for _,y in itertools.groupby(treeArray['j_index'])]
j_cIndex = np.array([],dtype='int8')

for i in x:
    new_jet_index = np.arange(i)
    j_cIndex = np.append(j_cIndex, new_jet_index)

In [34]:
split_index = np.array([])
for i in range(len(x)):
    if i%2470 == 0:
        split_index = np.append(split_index,np.sum(x[:i]))
split_index = np.append(split_index,np.sum(x))
split_index

array([      0.,  128852.,  256625.,  384513.,  514097.,  641712.,
        769858.,  896464., 1026061., 1156044., 1286398., 1417052.,
       1544928., 1672454., 1802252., 1931669., 2060728., 2188800.,
       2316946., 2444729., 2574941., 2702341., 2831864., 2960423.,
       3087693., 3214838., 3342852., 3471194., 3598272., 3727227.,
       3854143., 3981789., 4108865., 4236385., 4363616., 4492916.,
       4620150., 4747591., 4875242., 5004078., 5131613.])

In [9]:
# 暂存
with h5py.File("data/j_cIndex.h5",'w') as f1:
    f1.create_dataset('j_cIndex',data=j_cIndex)

In [35]:
ds_index = treeArray['index']
ds_JCPT = treeArray['j1_pt']
ds_JCETA = treeArray['j1_eta']
ds_JCPHI = treeArray['j1_phi']
ds_JCMASS = treeArray['j_mass']
ds_Nconstituents = treeArray['j_multiplicity']
ds_constituents_index = j_cIndex
ds_JCDELTAETA = treeArray['j1_etarot']
ds_JCDELTAPHI = treeArray['j1_phirot']
ds_PT = treeArray['j_pt']
ds_ETA = treeArray['j_eta']
# ds_PHI = treeArray['j_phi']

ds_label = np.vstack((treeArray['j_g'],treeArray['j_q'],treeArray['j_w'],treeArray['j_z'],treeArray['j_t'])).T

In [46]:
split_index = split_index.astype(int)

In [47]:
Label = ['index', 'JCPT', 'JCETA', 'JCPHI', 'JCMASS', 'Nconstituents', 'constituents_index', 'JCDELTAETA', 'JCDELTAPHI', 'PT', 'ETA','label']
for i in range(len(split_index)-1):
    with h5py.File('data/Particle_%d.h5'%i) as f2:
        for lb in Label:
            f2.create_dataset(lb, data=eval('ds_%s'%lb)[split_index[i]:split_index[i+1]])

D:\Softwares\Anacoda\envs\tf2\lib\site-packages\ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
##### old
# Label = ['index', 'JCPT', 'JCETA', 'JCPHI', 'JCMASS', 'Nconstituents', 'constituents_index', 'JCDELTAETA', 'JCDELTAPHI', 'PT', 'ETA','label']
# with h5py.File('data/data_Particle-net.h5','w') as f_new:
#     for lb in Label:
#         f_new.create_dataset(lb, data=eval('ds_%s'%lb))

# Start Here

In [7]:
##------------calculate jet constituents energy, if you already had this information don't use it.
def jce_np(jcpt, jceta, jcmass):
    jcp = jcpt*np.cosh(jceta)
    return (jcp**2+jcmass**2)**0.5


#-------- attach integers to your keys  this example is : 'index' -> 0-th data, 'JCPT' ->1-th data
# and 'index' is event index
# 'JCPT' is constituents transvers momentum (PT)
# 'JCETA' is constituents ETA
# 'JCPHI' is constituents angle around beam axis
# 'Ncontituents' is Number of constituents in the jet which the constitunt belong
# 'constiteunts_index' is constituents index in each jet ('constiteunts' is typo)
# 'JCDELTAETA' is constituents ETA related to jet axis
# 'JCDELTAPHI' is constituents ETA related to jet axis
# 'PT' is transvers momentum (PT) of jet which which the constitunt belong
# 'ETA' is ETA of jet which which the constitunt belong
# 'PHi' is PHI of jet which which the constitunt belong


_index, _jcpt, _jceta, _jcphi, _jcmass, _Ncons, _consindex, _jcdelteta, _jcdeltphi, _jpt, _jeta = (i for i in range(11))
Labels = ['index', 'JCPT', 'JCETA', 'JCPHI', 'JCMASS', 'Nconstituents', 'constituents_index', 'JCDELTAETA', 'JCDELTAPHI', 'PT', 'ETA']

##--------------------------------------------------h5py file to ParticleNet input structure
def h5_to_data(h5path):
    Data = {'mask':[], 'points':[], 'features':[]}
    f = h5py.File(h5path,'r')
    fc = np.array([f[lb][()] for lb in Labels])
    fc = fc.transpose((1,0))
    j0 = fc[0][_index]
    
    JCE = jce_np(fc[:,_jcpt], fc[:,_jceta], fc[:,_jcphi])
    logpt = np.log(fc[:,_jcpt])
    loge = np.log(JCE)
    relatpt = fc[:,_jcpt]/fc[:,_jpt]
    mask, features, points = np.zeros((100,1)), np.zeros((100,5)), np.zeros((100,2)) # prepare constituents list
    Nfc = len(fc)
    for j in range(len(fc)):
        if fc[j][_Ncons]>100:
            if j< Nfc-1:
                j0 = fc[j+1][_index]
            continue
        if fc[j][_index]!=j0:
            j0 = fc[j][_index]
            Data['mask'].append(mask)
            Data['points'].append(points)
            Data['features'].append(features)
            mask, features, points = np.zeros((100,1)), np.zeros((100,5)), np.zeros((100,2)) # prepare constituents list
            continue
        jc = int(fc[j][_consindex])
#         jce = JCE[j]
        
        points[jc] = np.array([fc[j][_jcdelteta], fc[j][_jcdeltphi] ])
        mask[jc] = logpt[j]
        features[jc] = np.array([logpt[j], loge[j], fc[j][_jcdelteta], fc[j][_jcdeltphi], relatpt[j]])
    return Data

##==========================================================================================
##----------merging 2 sample with ParticleNet input structure
def merging(gg,qq):
    total={}
    total['mask']=gg["mask"]+qq["mask"]
    total['features']=gg["features"]+qq["features"]
    total['points']=gg['points']+qq['points']
    return total
##--------- seperate inputs to training, validation and testing with ratio you give
def separatedata(features_list,y,rateval,ratetest):
    features_train, features_test, features_val={},{},{}
    from sklearn.model_selection import train_test_split
    mask = features_list["mask"]
    features = features_list["features"]
    points = features_list["points"]
    X_ind = [i for i in range(len(y))]
    X_train, X_ind, y_train, y_ind = train_test_split(X_ind, y, test_size=rateval+ratetest)
    N=int(len(X_ind)*rateval/(rateval+ratetest))
    X_val, X_test = X_ind[:N], X_ind[N:]
    y_val, y_test = y_ind[:N], y_ind[N:]
    features_train['mask']=np.array([mask[i] for i in X_train])
    features_train['features']=np.array([features[i] for i in X_train])
    features_train['points']=np.array([points[i] for i in X_train])
    
    features_test['mask']=np.array([mask[i] for i in X_test])
    features_test['features']=np.array([features[i] for i in X_test])
    features_test['points']=np.array([points[i] for i in X_test])
    
    features_val['mask']=np.array([mask[i] for i in X_val])
    features_val['features']=np.array([features[i] for i in X_val])
    features_val['points']=np.array([points[i] for i in X_val])
    
    return features_train, features_val, features_test,np.array(y_train), np.array(y_val), np.array(y_test)

In [ ]:
N = 40
h5Path = "data/data_Particle-net.h5"
Data = {'mask':[], 'points':[], 'features':[]}
for i in tqdm(range(N)):
    h5Path = "data/Particle_"+str(i)+".h5"
    data = h5_to_data(h5Path)
    Data = merging(Data,data)
print("check shape: ",Data['mask'][0].shape,Data['points'][0].shape,Data['features'][0].shape)

 25%|████████████████████▌                                                             | 10/40 [00:17<00:53,  1.77s/it]

In [ ]:
h5py.File(h5Path)['label'][()]

In [ ]:
# ## binary calssification: 
# y=[[0,1] for i in range(len(Data['mask']))]+[[1,0] for i in range(len(Data2['mask']))] 
# total = merging(Data, Data2)

y = h5py.File(h5Path)['label'][()]

# del Data
X_train, X_val, X_test, y_train, y_val, y_test = separatedata(Data,y,0.25,0.25)

In [ ]:
## check the shape
print([X_train[i][0].shape for i in ['mask', 'points', 'features']])

In [ ]:
import sys
sys.path.insert(0,'models')
from tf_keras_model import get_particle_net

In [ ]:
input_shapes={'points': X_train['points'][0].shape, 'features': X_train['features'][0].shape, 'mask': X_train['mask'][0].shape}
num_classes = 5
model = get_particle_net(num_classes, input_shapes)


In [ ]:
batch_size = 1024
epochs = 200

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train ,y_train,
          batch_size=batch_size,
          epochs=epochs, 
          validation_data=(X_val, y_val),
          shuffle=True ,
#           callbacks=callbacks
         )
model.evaluate(X_test,  y_test, verbose=2)